In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from helper import get_fng_index

In [ ]:
df = pd.read_csv("data/kucoin_btc_usdt1d.csv")

df = df.set_index(pd.DatetimeIndex(df.iloc[:,0].values)).iloc[:,1:]

df

In [ ]:
TRAIN_SPLIT = int(len(df["close"]) * 0.8)

train = df[['close']][:TRAIN_SPLIT]
test = df[['close']][TRAIN_SPLIT:]

plt.figure(figsize = (15,9))
plt.plot(train, label = "Train Data",color="blue")
plt.plot(test, label = "Test Data",color="red")
plt.title("Bitcoin Price",fontsize=18, fontweight='bold')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (USD)',fontsize=18)


In [ ]:
fng_df = get_fng_index()
fng_df

In [ ]:
result = pd.merge(df, fng_df, on='timestamp')

In [ ]:
model_lstm = tf.keras.models.load_model('saved_models/lstm')
model_lstm.summary()

In [ ]:
model_gru = tf.keras.models.load_model('saved_models/gru')
model_gru.summary()

In [ ]:
model_lstm_gru = tf.keras.models.load_model('saved_models/grulstm')
model_lstm_gru.summary()

In [ ]:
data = result[['close','volume','open','low','high','value_classification']]
num_features = data.shape[1]
scaler = MinMaxScaler()
norm_data = scaler.fit_transform(data.values)

past_history = 10
future_target = 0
split_rate = int(len(norm_data) * 0.8) # 80 percent


input_data = []
output_data = []

for i in range(past_history, len(norm_data) - future_target+1): 
    indices = range(i-past_history, i)

    input_data.append(np.reshape(norm_data[indices], (past_history, num_features)))
    try:
        output_data.append(norm_data[i+future_target][0])
    except:
        output_data.append(np.nan)

input_data,output_data = np.array(input_data), np.array(output_data)
x_train, x_test = input_data[:split_rate], input_data[split_rate:]
y_train, y_test = output_data[:split_rate], output_data[split_rate:]

# Choose Model:

model = model_lstm

original_train = y_train - scaler.min_[0]
original_train /= scaler.scale_[0]
original_train_lstm_fng = pd.DataFrame((original_train))

predictions_train = model.predict(x_train) - scaler.min_[0]
predictions_train /= scaler.scale_[0]
predictions_train_lstm_fng = pd.DataFrame((predictions_train))

plt.figure()

plt.plot(original_train_lstm_fng[0], 'blue', label='Bitcoin Price')
plt.plot(predictions_train_lstm_fng, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

original_test = y_test - scaler.min_[0]
original_test /= scaler.scale_[0]
original_test_lstm = pd.DataFrame((original_test))

predictions_test = model.predict(x_test) - scaler.min_[0]
predictions_test /= scaler.scale_[0]
predictions_test_lstm = pd.DataFrame((predictions_test))

plt.figure()

plt.plot(original_test_lstm[:], 'blue', label='Bitcoin Price')
plt.plot(predictions_test_lstm[:], 'red', label='Prediction of LSTM Model')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

print(mean_absolute_error(y_train, model.predict(x_train)))
print(mean_absolute_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1]),squared=False))

In [ ]:
prediction = pd.DataFrame({"actual price":original_test_lstm[0],"predicted":predictions_test_lstm[0]})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prediction)

In [ ]:
data = result[['close','volume','open','low','high','value_classification']]
num_features = data.shape[1]
scaler = MinMaxScaler()
norm_data = scaler.fit_transform(data.values)

past_history = 10
future_target = 0
split_rate = int(len(norm_data) * 0.8) # 80 percent


input_data = []
output_data = []

for i in range(past_history, len(norm_data) - future_target+1): 
    indices = range(i-past_history, i)

    input_data.append(np.reshape(norm_data[indices], (past_history, num_features)))
    try:
        output_data.append(norm_data[i+future_target][0])
    except:
        output_data.append(np.nan)

input_data,output_data = np.array(input_data), np.array(output_data)
x_train, x_test = input_data[:split_rate], input_data[split_rate:]
y_train, y_test = output_data[:split_rate], output_data[split_rate:]

# Choose Model:

model = model_gru

original_train = y_train - scaler.min_[0]
original_train /= scaler.scale_[0]
original_train2 = pd.DataFrame((original_train))

predictions_train = model.predict(x_train) - scaler.min_[0]
predictions_train /= scaler.scale_[0]
predictions_train2 = pd.DataFrame((predictions_train))

plt.figure()

plt.plot(original_train2[0], 'blue', label='Train Data')
plt.plot(predictions_train2, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

original_test = y_test - scaler.min_[0]
original_test /= scaler.scale_[0]
original_test_gru = pd.DataFrame((original_test))

predictions_test = model.predict(x_test) - scaler.min_[0]
predictions_test /= scaler.scale_[0]
predictions_test_gru = pd.DataFrame((predictions_test))

plt.figure()

plt.plot(original_test_gru[:], 'blue', label='Bitcoin Price')
plt.plot(predictions_test_gru[:], 'red', label='Prediction of GRU Model')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

print(mean_absolute_error(y_train, model.predict(x_train)))
print(mean_absolute_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1]),squared=False))

In [ ]:
prediction = pd.DataFrame({"actual price":original_test_gru[0],"predicted":predictions_test_gru[0]})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prediction)

In [ ]:
data = result[['close','volume','open','low','high','value_classification']]
num_features = data.shape[1]
scaler = MinMaxScaler()
norm_data = scaler.fit_transform(data.values)

past_history = 10
future_target = 0
split_rate = int(len(norm_data) * 0.8) # 80 percent


input_data = []
output_data = []

for i in range(past_history, len(norm_data) - future_target+1): 
    indices = range(i-past_history, i)

    input_data.append(np.reshape(norm_data[indices], (past_history, num_features)))
    try:
        output_data.append(norm_data[i+future_target][0])
    except:
        output_data.append(np.nan)

input_data,output_data = np.array(input_data), np.array(output_data)
x_train, x_test = input_data[:split_rate], input_data[split_rate:]
y_train, y_test = output_data[:split_rate], output_data[split_rate:]

# Choose Model:

model = model_lstm_gru

original_train = y_train - scaler.min_[0]
original_train /= scaler.scale_[0]
original_train3 = pd.DataFrame((original_train))

predictions_train = model.predict(x_train) - scaler.min_[0]
predictions_train /= scaler.scale_[0]
predictions_train3 = pd.DataFrame((predictions_train))

plt.figure()

plt.plot(original_train3[0], 'blue', label='Bitcoin Price')
plt.plot(predictions_train3, 'red', label='Prediction')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

original_test = y_test - scaler.min_[0]
original_test /= scaler.scale_[0]
original_test_lg = pd.DataFrame((original_test))

predictions_test = model.predict(x_test) - scaler.min_[0]
predictions_test /= scaler.scale_[0]
predictions_test_lg = pd.DataFrame((predictions_test))

plt.figure()

plt.plot(original_test_lg[:], 'blue', label='Bitcoin Price')
plt.plot(predictions_test_lg[:], 'red', label='Prediction of LSTM+GRU Model')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()
print(mean_absolute_error(y_train, model.predict(x_train)))
print(mean_absolute_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1])))
print(mean_squared_error(y_test[:-1], model.predict(x_test[:-1]),squared=False))

In [ ]:
prediction = pd.DataFrame({"actual price":original_test_lg[0],"predicted":predictions_test_lg[0]})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prediction)

In [ ]:
fig=plt.figure(figsize=(20,5))
s,f=0,-1
plt.plot(original_test_lg[s:f], 'blue', label='Bitcoin Price')
plt.plot(predictions_test_lstm[s:f], 'green', label='Prediction of LSTM Model')
plt.plot(predictions_test_lg[s:f], 'red', label='Prediction of proposed Model')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(20,5))
s,f=0,-1
plt.plot(original_test_lg[s:f], 'blue', label='Test Data')
plt.plot(predictions_test_gru[s:f], 'green', label='Prediction of GRU Model')
plt.plot(predictions_test_lg[s:f], 'red', label='Prediction of proposed Model')
plt.title("Bitcoin price")
plt.xlabel("Days")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

In [ ]:
n_groups = 349
means_frank = abs(original_test_lg[s:f][0] - predictions_test_lg[s:f][0])

# create plot
fig, ax = plt.subplots(figsize=(30, 5))
index = np.arange(n_groups)
bar_width = 0.3
opacity = 0.8

rects = plt.bar(index, means_frank, bar_width,
    alpha=opacity,
    color='b',
    label='LSTM+GRU')

plt.xlabel('Days')
plt.ylabel('Error')
plt.title('Price Error')
plt.xticks(index, range(n_groups))
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
n_groups = 349
means_frank = abs(original_test_gru[s:f][0] - predictions_test_gru[s:f][0])

# create plot
fig, ax = plt.subplots(figsize=(30, 5))
index = np.arange(n_groups)
bar_width = 0.3
opacity = 0.8

rects1 = plt.bar(index, means_frank, bar_width,
    alpha=opacity,
    color='b',
    label='GRU')


plt.xlabel('Days')
plt.ylabel('Error')
plt.title('Price Error')
plt.xticks(index, range(n_groups))
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
n_groups = 349
means_frank = abs(original_test_lstm[s:f][0] - predictions_test_lstm[s:f][0])

# create plot
fig, ax = plt.subplots(figsize=(30, 5))
index = np.arange(n_groups)
bar_width = 0.3
opacity = 0.8

rects1 = plt.bar(index, means_frank, bar_width,
    alpha=opacity,
    color='b',
    label='LSTM')

plt.xlabel('Days')
plt.ylabel('Error')
plt.title('Price Error')
plt.xticks(index, range(n_groups))
plt.legend()

plt.tight_layout()
plt.show()